In [2]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.static import teams

def generate_season_strings(start_year, end_year):
    """
    Convert numeric start/end years into NBA season strings.
    Example: start_year=2010, end_year=2025 → ["2010-11", "2011-12", ..., "2024-25"]
    """
    seasons = []
    for year in range(start_year, end_year):
        next_year = str(year + 1)[-2:]  # last two digits
        season_str = f"{year}-{next_year}"
        seasons.append(season_str)
    return seasons

def playoffs_binary_all_seasons(start_year, end_year, output_file):
    """
    Generate binary playoffs variable for all seasons in the given range,
    merge with team names, and export to one CSV file.
    """
    # Map team IDs to names once
    team_info = teams.get_teams()
    id_to_name = {team['id']: team['full_name'] for team in team_info}
    
    # Build season strings
    seasons = generate_season_strings(start_year, end_year)
    
    all_data = []
    
    for season in seasons:
        # Playoff teams
        playoff_games = leaguegamelog.LeagueGameLog(
            season=season,
            season_type_all_star='Playoffs'
        ).get_data_frames()[0]
        playoff_team_ids = playoff_games['TEAM_ID'].unique()
        
        # All teams (from regular season)
        regular_games = leaguegamelog.LeagueGameLog(
            season=season,
            season_type_all_star='Regular Season'
        ).get_data_frames()[0]
        all_team_ids = regular_games['TEAM_ID'].unique()
        
        # Build binary dictionary
        binary_playoffs = {tid: int(tid in playoff_team_ids) for tid in all_team_ids}
        
        # Append season data
        for tid in all_team_ids:
            all_data.append({
                'Season': season,
                'Team': id_to_name[tid],
                'Playoffs': binary_playoffs[tid]
            })
    
    # Create DataFrame
    df = pd.DataFrame(all_data)
    
    # Export to CSV
    df.to_csv(output_file, index=False)
    print(f"Saved combined file: {output_file}")

# Example usage:
playoffs_binary_all_seasons(start_year=2010, end_year=2025, output_file="nba_playoffs_binary_2010_2025.csv")


Saved combined file: nba_playoffs_binary_2010_2025.csv
